In [ ]:
# Transformers installation
! pip install transformers datasets evaluate seqeval accelerate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [ ]:
pretrained_model = "bert-base-cased"

In [ ]:
#Loading Data
import json
with open("main.json", encoding='utf-8') as fp:
    dataset = json.load(fp);

'''for i in range(len(dataset)):
    for j in range(len(dataset[i]['tokens'])):
        dataset[i]['tokens'][j] = dataset[i]['tokens'][j].upper()'''
dataset[0]

In [ ]:

label2id = {
    "O":0,
    "B-no":1,
    "I-no":2,
    "B-class":3,
    "I-class":4,
    "B-method":5,
    "I-method":6,
    "B-attribute":7,
    "I-attribute":8,
    "B-association":9,
    "I-association":10,
    "B-generalization":11,
    "I-generalization":12,
}

id2label = {
    0:"O",
    1:"B-no",
    2:"I-no",
    3:"B-class",
    4:"I-class",
    5:"B-method",
    6:"I-method",
    7:"B-attribute",
    8:"I-attribute",
    9:"B-association",
    10:"I-association",
    11:"B-generalization",
    12:"I-generalization",
}

In [ ]:
for i in range(len(dataset)):
    dataset[i]["ner_tags"] = [label2id[j] for j in dataset[i]["ner_tags"]]

dataset[0]

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset,test_size=0.15)
print(len(train_set),len(test_set))

In [ ]:
from datasets import IterableDataset

def gen_train():
    for i in train_set:
        yield i

def gen_test():
    for i in test_set:
        yield i

train_dataset = IterableDataset.from_generator(gen_train)
test_dataset = IterableDataset.from_generator(gen_test)

In [ ]:
label_list = list(label2id.keys())
label_list

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [10]:
# example = wnut["train"][0]
example = dataset[0]
print(example)
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

{'id': '0', 'ner_tags': [1, 5, 1, 5, 3, 1], 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}


['[CLS]',
 '3',
 '.',
 '1',
 '.',
 '1',
 'Start',
 'and',
 'Stop',
 'Di',
 '##ag',
 '##nos',
 '##tics',
 'Use',
 'Case',
 '[SEP]']

In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
# tokenized_dataset = list(map(tokenize_and_align_labels, dataset))
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

In [13]:
for i in tokenized_test_dataset:
    print(i)

{'id': '2622', 'ner_tags': [1, 0, 1, 2], 'tokens': ['Name', ':', 'real-time', 'processing'], 'input_ids': [101, 10208, 131, 1842, 118, 1159, 6165, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 1, 0, 1, -100, -100, 2, -100]}
{'id': '2319', 'ner_tags': [0, 0, 7], 'tokens': ['1.2.8', '.', 'End Date'], 'input_ids': [101, 122, 119, 123, 119, 129, 119, 5135, 14265, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, -100, -100, -100, -100, 0, 7, -100, -100]}
{'id': '1887', 'ner_tags': [0, 0, 0, 3, 0, 3, 0, 0, 0, 3], 'tokens': ['4.6.1.3', '.', 'Parent', 'Department', 'or', 'Category', 'of', 'the', 'new', 'Category'], 'input_ids': [101, 125, 119, 127, 119, 122, 119, 124, 119, 19585, 17759, 1951, 1137, 13696, 1104, 1103, 1207, 13696, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 

In [14]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


{'id': '434', 'ner_tags': [1, 5, 1, 0, 1, 2, 2, 2, 2, 0, 1, 5, 1, 0, 0, 1, 0], 'tokens': ['4.1.3.8b', 'Exit', 'from', 'Shunting', 'shall', 'take', 'place', 'when', 'the', 'driver', 'selects', 'exit', 'from', 'shunting', '. (', 'M', ')'], 'input_ids': [101, 125, 119, 122, 119, 124, 119, 129, 1830, 19588, 1121, 23274, 26728, 1403, 4103, 1321, 1282, 1165, 1103, 3445, 8247, 1116, 6300, 1121, 188, 17315, 1916, 119, 113, 150, 114, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 1, -100, -100, -100, -100, -100, -100, -100, 5, 1, 0, -100, -100, 1, 2, 2, 2, 2, 0, 1, -100, 5, 1, 0, -100, -100, 0, -100, 1, 0, -100]}
{'id': '907', 'ner_tags': [0, 0, 3, 5, 6], 'tokens': ['3', '.', 'User', 'selects', 'Cancel'], 'input_ids': [101, 124, 119, 11696, 1197, 8247, 1116, 2825, 18389, 102], 'token_type_

In [15]:
import evaluate

seqeval = evaluate.load("seqeval")

In [16]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "accuracy": results["overall_accuracy"],
    }

In [17]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    pretrained_model, num_labels=13, id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    max_steps = 250,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [19]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

In [ ]:
text = ["Cities and towns have airports."]

In [ ]:
from transformers import pipeline

classifier = pipeline("ner", model=model, tokenizer=tokenizer, device='cuda:0')
classifier(text)

In [ ]:
'''from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_wnut_model")
inputs = tokenizer(text, return_tensors="pt")'''

In [ ]:
'''from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("stevhliu/my_awesome_wnut_model")
with torch.no_grad():
    logits = model(**inputs).logits'''

In [ ]:
'''predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class'''